データの読み込みと前処理を行うためのnotebookです。  
モデルの学習と予測にはここで処理をかけたデータを利用するようにして下さい。

## 必要なライブラリのimport

In [1]:
import warnings
import time
import sys
import datetime
import os

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
warnings.simplefilter(action='ignore', category=FutureWarning)

## データの読み込み

In [2]:
def reduce_mem_usage(df, verbose=True):
    """
    データフレームのメモリ使用量を減らす。

    Parameters
    ----------
    df : pd.DataFrame
        メモリ使用量を削減したいデータフレーム。
    verbose : bool, optional
        メモリ使用量の削減結果を出力するかどうか（デフォルトは True）。

    Returns
    -------
    pd.DataFrame
        メモリ使用量が削減されたデータフレーム。
    """

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


def binarize(df):
    """
    指定された列を二値化する。

    Parameters
    ----------
    df : pd.DataFrame
        二値化対象のデータフレーム。

    Returns
    -------
    pd.DataFrame
        二値化されたデータフレーム。
    """

    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y': 1, 'N': 0})
    return df


# 追加
def binarize_bool(df,bool_col):
    """
    指定された列を二値化する。

    Parameters
    ----------
    df : pd.DataFrame
        二値化対象のデータフレーム。

    Returns
    -------
    pd.DataFrame
        二値化されたデータフレーム。
    """

    for col in bool_col:
        if col in df.columns:  # 列が存在することを確認
            df[col] = df[col].map({True: 1, False: 0})
        else:
            print(f"Warning: Column '{col}' does not exist in the DataFrame.")
    return df

def read_data(input_file):
    """
    指定されたファイルからデータを読み込み、前処理を行う。

    Parameters
    ----------
    input_file : str
        読み込むデータファイルのパス。

    Returns
    -------
    pd.DataFrame
        前処理されたデータフレーム。
    """

    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    # 基準日(2018-02-01)からの日付差(データが存在する最終日らしい)
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [3]:
'''
# windows
if os.name == 'nt':
    path = '../../../data/elo-merchant-category-recommendation/'
else:
    if 'KAGGLE_DATA_PROXY_TOKEN' in os.environ.keys():
        path = '/kaggle/input/elo-merchant-category-recommendaton/'

'''

In [ ]:
'''
train_path = os.path.join(path, 'train.csv')
test_path = os.path.join(path,'test.csv')
outlier_path = os.path.join(path,'outlier_flag.csv')
new_transactions_path = os.path.join(path,'new_merchant_transactions.csv')
historical_transactions_path = os.path.join(path,'historical_transactions.csv')

train = read_data(train_path)
test = read_data(test_path)
new_transactions = pd.read_csv(new_transactions_path,
                               parse_dates=['purchase_date'])

historical_transactions = pd.read_csv(historical_transactions_path,
                                      parse_dates=['purchase_date'])

# N,Yを0,1に変換
historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)
'''

In [3]:
train = read_data('../data/raw/train.csv')
test = read_data('../data/raw/test.csv')

new_transactions = pd.read_csv('../data/raw/new_merchant_transactions.csv',
                               parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('../data/raw/historical_transactions.csv',
                                      parse_dates=['purchase_date'])

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [19]:
train_hozon = train
test_hozon = test
historical_transactions_hozon = historical_transactions
new_transactions_hozon = new_transactions

## 特徴量作成

In [4]:
# mode関数を追加
def mode(series):
    return series.mode().iloc[0]  # modeが複数ある場合、最初のものを選ぶ

def calculate_month_diff(transactions):
    """
    purchase_dateとmonth_lagを基にmonth_diffを計算する。

    Parameters
    ----------
    transactions : pd.DataFrame
        取引データのデータフレーム。

    Returns
    -------
    pd.DataFrame
        month_diff列が追加されたデータフレーム。
    """
    current_date = pd.Timestamp('2018-02-01') # 今日…今日？？？-> 基準日(2018-02-01)にしてみた
    transactions['month_diff'] = ((current_date - transactions['purchase_date']).dt.days) // 30
    transactions['month_diff'] += transactions['month_lag']
    return transactions


def encode_categorical_columns(df, columns):
    """
    指定されたカテゴリカル列をワンホットエンコーディングする。

    Parameters
    ----------
    df : pd.DataFrame
        エンコード対象のデータフレーム。
    columns : list of str
        エンコードするカテゴリカル列のリスト。

    Returns
    -------
    pd.DataFrame
        ワンホットエンコードされたデータフレーム。
    """
    return pd.get_dummies(df, columns=columns)


def reduce_mem_usage(df, verbose=True):
    """
    データフレームのメモリ使用量を減らす。

    Parameters
    ----------
    df : pd.DataFrame
        メモリ使用量を削減したいデータフレーム。
    verbose : bool, optional
        メモリ使用量の削減結果を出力するかどうか（デフォルトは True）。

    Returns
    -------
    pd.DataFrame
        メモリ使用量が削減されたデータフレーム。
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

'''
def aggregate_transactions(history):
    """
    取引データを集計する。

    Parameters
    ----------
    history : pd.DataFrame
        取引データのデータフレーム。

    Returns
    -------
    pd.DataFrame
        集計されたデータフレーム。
    """
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).astype(np.int64) * 1e-9
    # modeを追加
    agg_func = {
        'category_1': ['sum', 'mean',mode],
        'category_2_1.0': ['mean',mode],
        'category_2_2.0': ['mean',mode],
        'category_2_3.0': ['mean',mode],
        'category_2_4.0': ['mean',mode],
        'category_2_5.0': ['mean',mode],
        'category_3_A': ['mean',mode],
        'category_3_B': ['mean',mode],
        'category_3_C': ['mean',mode],
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std',mode],
        'installments': ['sum', 'mean', 'max', 'min', 'std',mode],
        'purchase_month': ['mean', 'max', 'min', 'std',mode],
        # ptpはmaxとminの差らしい
        'purchase_date': [np.ptp, 'min', 'max',mode],
        'month_lag': ['mean', 'max', 'min', 'std',mode],
        # month_diffの平均、最大、最小をauthorized_flagごとに集計
        'month_diff': ['mean', 'max' , 'min']
    }

    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)

    df = (history.groupby('card_id')
            .size()
            .reset_index(name='transactions_count'))
    
    ##### カードIDごとに最も遅い取引日の金額を抽出
    # カードIDごとに日付が最も遅いレコードの金額を保持する辞書を作成する
    latest_amounts = {}

    for record in history:
        card_id = record['card_id']
        date = record['date']
        amount = record['amount']

        # カードIDごとに最も遅い日付の金額を更新する
        if card_id not in latest_amounts or date > latest_amounts[card_id]['date']:
            latest_amounts[card_id] = {'date': date, 'amount': amount}

    df['latest_amount'] = history['card_id'].map(lambda x: latest_amounts[x]['amount'])
    #####

    ##### カードIDごとに取引金額が最も多い日を抽出
    # カードIDごとに金額が最も多いレコードの日付を保持する辞書を作成する
    most_amounts_date = {}

    for record in history:
        card_id = record['card_id']
        date = record['date']
        amount = record['amount']

        # カードIDごとに最も遅い日付の金額を更新する
        if card_id not in latest_amounts or amount > most_amounts_date[card_id]['amount']:
            most_amounts_date[card_id] = {'date': date, 'amount': amount}

    df['most_amounts_date'] = history['card_id'].map(lambda x: most_amounts_date[x]['date'])
    #####

    agg_history = pd.merge(df, agg_history, on='card_id', how='left')

    return agg_history
'''

def aggregate_per_month(history):
    """
    月ごとの取引データを集計する。

    Parameters
    ----------
    history : pd.DataFrame
        取引データのデータフレーム。

    Returns
    -------
    pd.DataFrame
        月ごとに集計されたデータフレーム。
    """
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
        'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
        'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
    }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)

    return final_group


def successive_aggregates(df, field1, field2):
    """
    指定されたフィールドを基に連続集計を行う。
    要は2値グループバイしたmeanとminとmaxとstdを出す関数ってことね
    Parameters
    ----------
    df : pd.DataFrame
        取引データのデータフレーム。
    field1 : str
        集計の基準となるフィールド。
    field2 : str
        集計されるフィールド。

    Returns
    -------
    pd.DataFrame
        連続集計されたデータフレーム。
    """
    t = df.groupby(['card_id', field1])[field2].mean()
    u = pd.DataFrame(t).reset_index().groupby('card_id')[field2].agg(['mean', 'min', 'max', 'std'])
    u.columns = [field1 + '_' + field2 + '_' + col for col in u.columns.values]
    u.reset_index(inplace=True)
    return u


In [5]:

def aggregate_transactions(history):
    """
    取引データを集計する。

    Parameters
    ----------
    history : pd.DataFrame
        取引データのデータフレーム。

    Returns
    -------
    pd.DataFrame
        集計されたデータフレーム。
    """
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).astype(np.int64) * 1e-9
    
    # mode関数の定義
    def mode_func(x):
        return mode(x)[0][0] if len(x) > 0 else np.nan
    
    # 集計関数の定義
    agg_func = {
        'category_1': ['sum', 'mean',mode],
        'category_2_1.0': ['mean',mode],
        'category_2_2.0': ['mean',mode],
        'category_2_3.0': ['mean',mode],
        'category_2_4.0': ['mean',mode],
        'category_2_5.0': ['mean',mode],
        'category_3_A': ['mean',mode],
        'category_3_B': ['mean',mode],
        'category_3_C': ['mean',mode],
        'merchant_id': ['nunique'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'city_id': ['nunique'],
        'subsector_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std',mode],
        'installments': ['sum', 'mean', 'max', 'min', 'std',mode],
        'purchase_month': ['mean', 'max', 'min', 'std',mode],
        # ptpはmaxとminの差らしい
        'purchase_date': [np.ptp, 'min', 'max',mode],
        'month_lag': ['mean', 'max', 'min', 'std',mode],
        # month_diffの平均、最大、最小をauthorized_flagごとに集計
        'month_diff': ['mean', 'max' , 'min']
    }

    # 取引データの集計
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)

    # カードIDごとの取引数の集計
    df = history.groupby('card_id').size().reset_index(name='transactions_count')
    
    # カードIDごとに最も遅い取引の金額を抽出
    latest_transactions = history.loc[history.groupby('card_id')['purchase_date'].idxmax()]
    latest_transactions = latest_transactions[['card_id', 'purchase_date', 'purchase_amount']]
    df = pd.merge(df, latest_transactions, on='card_id', how='left')
    df.rename(columns={'purchase_amount': 'latest_amount'}, inplace=True)

    # カードIDごとに取引金額が最も多い日を抽出
    most_amount_transactions = history.loc[history.groupby('card_id')['purchase_amount'].idxmax()]
    most_amount_transactions = most_amount_transactions[['card_id', 'purchase_date']]
    df = pd.merge(df, most_amount_transactions, on='card_id', how='left')
    df.rename(columns={'purchase_date': 'most_amounts_date'}, inplace=True)

    # 集計結果とマージして返す
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')

    return agg_history

In [6]:
# データ準備
historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'])
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])

# 月の差を計算
historical_transactions = calculate_month_diff(historical_transactions)
new_transactions = calculate_month_diff(new_transactions)

# カテゴリカル列をワンホットエンコーディング
historical_transactions = encode_categorical_columns(historical_transactions, ['category_2', 'category_3'])
new_transactions = encode_categorical_columns(new_transactions, ['category_2', 'category_3'])

# メモリ使用量の削減
historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)

# authorized_flagの平均を計算
agg_fun = {'authorized_flag': ['mean']}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

# authorized_flagに基づいてデータを分割
authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]

# purchase_month列を追加
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month

# データの集計
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]

# # rankの設定
# history['month_rank'] = historical_transactions.groupby(['card_id'])['month_lag'].rank(method='dense',ascending=False)
# history['date_rank'] = historical_transactions.groupby(['card_id'])['purchase_date'].rank(method='dense',ascending=False)


# authorized = aggregate_transactions(authorized_transactions)
# authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]

# new = aggregate_transactions(new_transactions)
# new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]

# # 月ごとのデータの集計
# final_group = aggregate_per_month(authorized_transactions)

# # 連続集計(new_transactionにくっつけてく)
# additional_fields = successive_aggregates(new_transactions, 'category_1', 'purchase_amount')
# additional_fields = pd.merge(additional_fields, successive_aggregates(new_transactions, 'installments', 'purchase_amount'), on='card_id', how='left')
# additional_fields = pd.merge(additional_fields, successive_aggregates(new_transactions, 'city_id', 'purchase_amount'), on='card_id', how='left')
# additional_fields = pd.merge(additional_fields, successive_aggregates(new_transactions, 'category_1', 'installments'), on='card_id', how='left')

Mem. usage decreased to 1332.66 Mb (57.1% reduction)
Mem. usage decreased to 86.12 Mb (58.9% reduction)


In [7]:
len(historical_transactions)

2516909

In [8]:
len(history)

275701

In [9]:
# rankの設定
df = historical_transactions.groupby(['card_id'])['month_lag'].rank(method='dense',ascending=False)
df2 = historical_transactions.groupby(['card_id'])['purchase_date'].rank(method='dense',ascending=False)


In [10]:
len(df)

2516909

In [11]:
a = history.copy()

In [13]:
len(a)

275701

In [14]:
a['month_rank'] = historical_transactions.groupby(['card_id'])['month_lag'].rank(method='dense',ascending=False)
len(a)

275701

In [17]:
sum(a['month_rank'].isna())

257232

In [23]:
# データの結合
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test = pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

train = pd.merge(train, additional_fields, on='card_id', how='left')
test = pd.merge(test, additional_fields, on='card_id', how='left')

In [24]:
# rate特徴量の追加
train['rate_new/hist_purchase_date_max'] = train['new_purchase_date_max']/train['hist_purchase_date_max']
test['rate_new/hist_purchase_date_max'] = test['new_purchase_date_max']/test['hist_purchase_date_max']

train['rate_new/hist_month_diff_mean'] = train['new_month_diff_mean']/train['hist_month_diff_mean']
test['rate_new/hist_month_diff_mean'] = test['new_month_diff_mean']/test['hist_month_diff_mean']

train['rate_new/hist_purchase_amount_sum'] = train['new_purchase_amount_sum']/train['hist_purchase_amount_sum']
test['rate_new/hist_purchase_amount_sum'] = test['new_purchase_amount_sum']/test['hist_purchase_amount_sum']

train['rate_new/hist_purchase_amount_max'] = train['new_purchase_amount_max']/train['hist_purchase_amount_max']
test['rate_new/hist_purchase_amount_max'] = test['new_purchase_amount_max']/test['hist_purchase_amount_max']

train['rate_new/hist_purchase_amount_mean'] = train['new_purchase_amount_mean']/train['hist_purchase_amount_mean']
test['rate_new/hist_purchase_amount_mean'] = test['new_purchase_amount_mean']/test['hist_purchase_amount_mean']

In [25]:
# 追加　True,Falseの変換
bool_col = ['hist_category_2_1.0_mode','hist_category_2_2.0_mode','hist_category_2_3.0_mode','hist_category_2_4.0_mode','hist_category_2_5.0_mode','hist_category_3_A_mode','hist_category_3_B_mode','hist_category_3_C_mode','new_category_2_1.0_mode','new_category_2_2.0_mode','new_category_2_3.0_mode','new_category_2_4.0_mode','new_category_2_5.0_mode','new_category_3_A_mode','new_category_3_B_mode','new_category_3_C_mode']

train = binarize_bool(df=train,bool_col=bool_col)
test = binarize_bool(df=test,bool_col=bool_col)

## 前処理終了後のデータの保存
- 基本的にモデルの学習・ハイパーパラメータチューニングを行う際にはここで作成した同じデータを使い回して下さい。
- 適宜前処理を変更した場合はファイル名を変えるなどして管理して下さい。

In [27]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,...,city_id_purchase_amount_std,category_1_installments_mean,category_1_installments_min,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,245,13.0,1.514385e+09,-0.728876,...,0.155779,0.000000,0.000000,0.00,NaN,1.007010,1.000000,1.545068,0.685639,0.873299
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,396,11.0,1.514467e+09,-0.734887,...,NaN,1.000000,1.000000,1.00,NaN,1.005233,0.000000,3.878816,-0.361152,7.111163
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,549,2.0,1.492801e+09,-0.701453,...,NaN,0.000000,0.000000,0.00,NaN,1.021527,1.000000,0.522937,1.098320,1.045874
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,153,NaN,NaN,NaN,...,0.001554,0.833333,0.666667,1.00,0.235702,NaN,NaN,NaN,NaN,NaN
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,92,5.0,1.519759e+09,6.992617,...,0.150666,1.220588,0.941176,1.50,0.395148,1.003410,0.861111,-0.979037,0.062689,-0.135977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,-2.740821,153,4.0,1.518114e+09,-0.679289,...,NaN,0.000000,0.000000,0.00,NaN,1.001734,1.000000,0.252686,1.062993,1.010742
201913,2015-10-01,C_ID_1314773c0b,3,1,1,0.312917,854,7.0,1.511473e+09,-0.707869,...,NaN,0.000000,0.000000,0.00,NaN,1.006668,1.166667,0.162604,1.295841,1.138231
201914,2017-08-01,C_ID_7666735b3d,4,3,0,0.093494,184,6.0,1.513959e+09,-0.637920,...,0.073779,2.000000,1.000000,3.00,1.414214,1.005468,1.000000,0.191072,-0.289383,0.382143
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,-4.676589,580,2.0,1.509124e+09,-0.723632,...,NaN,0.000000,0.000000,0.00,NaN,1.003478,1.066667,-1.325778,-3.120113,-0.530311


In [28]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,...,city_id_purchase_amount_std,category_1_installments_mean,category_1_installments_min,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,306.0,24.0,1.513186e+09,-0.476731,1.503441e+09,...,0.182864,1.666667,1.666667,1.666667,NaN,1.004401,1.000000,0.140001,-6.388255,1.120009
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,396.0,1.0,1.494079e+09,-0.731881,1.494079e+09,...,0.014353,1.142857,1.000000,1.285714,0.202031,1.020192,-inf,8.121995,0.691844,0.902444
2,2017-08-01,C_ID_b709037bc5,5,1,1,184.0,4.0,1.517593e+09,0.214624,1.515681e+09,...,1.151811,5.500000,1.000000,10.000000,6.363961,1.002210,0.500000,0.033132,0.358015,0.066264
3,2017-12-01,C_ID_d27d835a9f,2,1,0,62.0,NaN,NaN,NaN,NaN,...,0.083171,6.944444,1.888889,12.000000,7.149635,NaN,NaN,NaN,NaN,NaN
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,793.0,23.0,1.514992e+09,-0.596643,1.491330e+09,...,1.288398,0.833333,0.833333,0.833333,NaN,1.005639,3.833333,-0.775375,39.511246,-2.972271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,2,2,0,123.0,3.0,1.518957e+09,-0.705870,1.513517e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123619,2017-09-01,C_ID_75ace375ae,3,1,1,153.0,NaN,NaN,NaN,NaN,...,0.097762,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
123620,2016-09-01,C_ID_21d56d950c,5,1,1,518.0,1.0,1.506011e+09,0.369802,1.506011e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123621,2017-06-01,C_ID_6c46fc5a9d,2,1,0,245.0,19.0,1.506362e+09,-0.737892,1.500295e+09,...,0.062056,1.300000,0.600000,2.000000,0.989949,1.003513,1.091954,0.306193,0.999174,0.969611


In [29]:
train.to_csv('../data/processed/processed20240621_train.csv',index=None)
test.to_csv('../data/processed/processed20240621_test.csv',index=None)

フラグ追加最終説明変数

In [4]:
# trainデータの読み込み
train = pd.read_csv('../data/processed/process20240624-2_train.csv')
# train_trueflag = pd.read_csv('../data/processed/processed20240620-1_train.csv')
train_flpr = pd.read_csv('../data/processed/train_with_outlier_flag_probability_16.csv')

In [5]:
train.head()

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,...,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag_ver_3,outlier_probability_ver_3,outlier_flag_ver_4,outlier_probability_ver_4,target_outlier_flag,target
0,0,2017-06-01,C_ID_92a2005557,5,2,1,245,13.0,1.514385e+09,-0.728876,...,1.000000,1.545068,0.685639,0.873299,0,1.133430e-11,0,4.220536e-34,0,-0.820283
1,1,2017-01-01,C_ID_3d0044924f,4,1,0,396,11.0,1.514467e+09,-0.734887,...,0.000000,3.878816,-0.361152,7.111163,0,1.375640e-04,0,2.020771e-16,0,0.392913
2,2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,2.0,1.492801e+09,-0.701453,...,1.000000,0.522937,1.098320,1.045874,0,1.351713e-07,0,6.510316e-27,0,0.688056
3,3,2017-09-01,C_ID_186d6a6901,4,3,0,153,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,3.634273e-06,0,5.725371e-15,0,0.142495
4,4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,5.0,1.519759e+09,6.992617,...,0.861111,-0.979037,0.062689,-0.135977,0,6.830008e-11,0,0.000000e+00,0,-0.159749


In [10]:
train.head()

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,...,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag_ver_3,outlier_probability_ver_3,outlier_flag_ver_4,outlier_probability_ver_4,target_outlier_flag,target
0,0,2017-06-01,C_ID_92a2005557,5,2,1,245,13.0,1.514385e+09,-0.728876,...,1.000000,1.545068,0.685639,0.873299,0,1.133430e-11,0,4.220536e-34,0,-0.820283
1,1,2017-01-01,C_ID_3d0044924f,4,1,0,396,11.0,1.514467e+09,-0.734887,...,0.000000,3.878816,-0.361152,7.111163,0,1.375640e-04,0,2.020771e-16,0,0.392913
2,2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,2.0,1.492801e+09,-0.701453,...,1.000000,0.522937,1.098320,1.045874,0,1.351713e-07,0,6.510316e-27,0,0.688056
3,3,2017-09-01,C_ID_186d6a6901,4,3,0,153,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,3.634273e-06,0,5.725371e-15,0,0.142495
4,4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,5.0,1.519759e+09,6.992617,...,0.861111,-0.979037,0.062689,-0.135977,0,6.830008e-11,0,0.000000e+00,0,-0.159749


In [6]:
train_flpr.head()

,Unnamed: 0,first_active_month,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,...,category_1_installments_min,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag,outlier_probability
0,0,0.709646,1.832981,0.571061,1.076057,-0.708629,0.133667,0.644533,-0.008712,-0.225941,...,-0.810891,-0.739141,-0.096850,0.398512,0.003247,0.004150,0.007263,0.003354,0,7.820608e-08
1,1,0.194557,0.922036,-0.866564,-0.929319,-0.193256,-0.004705,0.654707,-0.008724,0.417420,...,0.191335,-0.011081,-0.096850,-0.091161,-2.915103,0.025842,-0.014192,0.157801,0,4.888127e-05
2,2,-0.327355,-0.899855,0.571061,-0.929319,0.328944,-0.627381,-2.031314,-0.008658,-1.523181,...,-0.810891,-0.739141,-0.096850,4.399107,0.003247,-0.005351,0.015721,0.007627,0,8.575983e-06
3,3,1.023475,0.922036,2.008686,-0.929319,-1.022632,-0.350636,0.583098,-0.008603,0.469290,...,-0.142740,-0.011081,-0.467275,-0.151676,0.003247,-0.004063,0.010244,0.006121,0,4.145552e-05
4,4,1.231557,-1.810800,2.008686,-0.929319,-1.230829,-0.419822,1.310776,0.006614,1.851973,...,0.132381,0.352949,-0.185371,-0.593574,-0.402080,-0.019311,-0.005505,-0.021635,0,9.206194e-06


In [14]:
train_flpr_just = train_flpr[['outlier_flag', 'outlier_probability']]
train_flpr_just.columns = ['outlier_flag_ver_5', 'outlier_probability_ver_5']
ketsu = pd.concat([train, train_flpr_just], axis = 1).drop(train.columns[0],axis=1)

In [15]:
ketsu

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,...,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag_ver_3,outlier_probability_ver_3,outlier_flag_ver_4,outlier_probability_ver_4,target_outlier_flag,target,outlier_flag_ver_5,outlier_probability_ver_5
0,2017-06-01,C_ID_92a2005557,5,2,1,245,13.0,1.514385e+09,-0.728876,1.500131e+09,...,0.685639,0.873299,0,1.133430e-11,0,4.220536e-34,0,-0.820283,0,7.820608e-08
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,11.0,1.514467e+09,-0.734887,1.506207e+09,...,-0.361152,7.111163,0,1.375640e-04,0,2.020771e-16,0,0.392913,0,4.888127e-05
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,2.0,1.492801e+09,-0.701453,1.487878e+09,...,1.098320,1.045874,0,1.351713e-07,0,6.510316e-27,0,0.688056,0,8.575983e-06
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,NaN,NaN,NaN,NaN,...,NaN,NaN,0,3.634273e-06,0,5.725371e-15,0,0.142495,0,4.145552e-05
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,5.0,1.519759e+09,6.992617,1.519756e+09,...,0.062689,-0.135977,0,6.830008e-11,0,0.000000e+00,0,-0.159749,0,9.206194e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,153,4.0,1.518114e+09,-0.679288,1.504812e+09,...,1.062993,1.010742,0,3.141244e-07,0,2.215855e-16,0,-2.740821,0,8.127339e-08
201913,2015-10-01,C_ID_1314773c0b,3,1,1,854,7.0,1.511473e+09,-0.707869,1.488832e+09,...,1.295841,1.138231,0,5.939084e-08,0,2.244641e-20,0,0.312917,0,7.121939e-07
201914,2017-08-01,C_ID_7666735b3d,4,3,0,184,6.0,1.513959e+09,-0.637921,1.511810e+09,...,-0.289383,0.382143,0,8.464170e-05,0,8.320879e-09,0,0.093494,0,2.557519e-05
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,580,2.0,1.509124e+09,-0.723632,1.509124e+09,...,-3.120113,-0.530311,0,1.778938e-06,0,8.359521e-16,0,-4.676589,0,1.645613e-04


In [17]:
# testデータの読み込み
test = pd.read_csv('../data/processed/process20240624-2_test.csv')
# train_trueflag = pd.read_csv('../data/processed/processed20240620-1_train.csv')
test_flpr = pd.read_csv('../data/processed/test_with_outlier_flag_probability_16.csv')

In [19]:
test.head()

,Unnamed: 0,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,...,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag_ver_1,outlier_flag_ver_2,outlier_flag_ver_3,outlier_probability_ver_3,outlier_flag_ver_4,outlier_probability_ver_4
0,0,2017-04-01,C_ID_0ab67a22ab,3,3,1,306.0,24.0,1.513186e+09,-0.476731,...,1.000000,0.140001,-6.388255,1.120009,0,0,0,2.074851e-03,0,5.672927e-06
1,1,2017-01-01,C_ID_130fd0cbdd,2,3,0,396.0,1.0,1.494079e+09,-0.731881,...,-inf,8.121995,0.691844,0.902444,0,0,0,9.908335e-06,0,2.005090e-17
2,2,2017-08-01,C_ID_b709037bc5,5,1,1,184.0,4.0,1.517593e+09,0.214624,...,0.500000,0.033132,0.358015,0.066264,0,0,0,1.796332e-06,0,4.571899e-10
3,3,2017-12-01,C_ID_d27d835a9f,2,1,0,62.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,0,0,3.490204e-11,0,1.179702e-35
4,4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,793.0,23.0,1.514992e+09,-0.596643,...,3.833333,-0.775375,39.511246,-2.972271,0,0,0,3.157762e-16,0,0.000000e+00


In [18]:
test_flpr

,Unnamed: 0,first_active_month,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,...,category_1_installments_min,category_1_installments_max,category_1_installments_std,rate_new/hist_purchase_date_max,rate_new/hist_month_diff_mean,rate_new/hist_purchase_amount_sum,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag,outlier_probability
0,0,0.501563,0.011090,2.008686,1.076057,-0.500432,0.894715,0.495806,-0.008212,0.124577,...,0.859486,0.474292,-0.050547,-0.320466,0.003247,-0.008910,-0.137720,0.009462,0,1.534506e-04
1,1,0.194557,-0.899855,2.008686,-0.929319,-0.193256,-0.696567,-1.872939,-0.008718,-0.866706,...,0.191335,0.196936,-0.526807,4.031115,0.003247,0.065282,0.007390,0.004076,0,3.860746e-05
2,2,0.917728,1.832981,-0.866564,1.076057,-0.916827,-0.489009,1.042210,-0.006840,1.420462,...,0.191335,6.541456,10.367645,-0.924253,-1.455928,-0.009903,0.000548,-0.016628,0,2.737026e-04
3,3,1.333892,-0.899855,-0.866564,-0.929319,-1.333221,-0.350636,0.588952,-0.008600,0.477846,...,1.082203,7.997576,11.756737,-0.150825,0.003247,-0.004115,0.010272,0.006140,0,2.927275e-07
4,4,-1.159684,1.832981,-0.866564,1.076057,1.161733,0.825529,0.719779,-0.008450,-1.157677,...,0.024298,-0.132425,-0.050547,0.020714,8.271902,-0.017418,0.803012,-0.091861,0,1.227157e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,123618,1.125810,-0.899855,0.571061,-0.929319,-1.125024,-0.558195,1.211319,-0.008667,1.191373,...,-0.476816,-0.375111,-0.050547,-0.150825,0.003247,-0.004115,0.010272,0.006140,0,1.652304e-07
123619,123619,1.023475,0.011090,-0.866564,1.076057,-1.022632,-0.350636,0.588952,-0.008600,0.477846,...,-0.810891,-0.739141,-0.050547,-0.150825,0.003247,-0.004115,0.010272,0.006140,0,4.465608e-07
123620,123620,-0.221608,1.832981,-0.866564,1.076057,0.223139,-0.696567,-0.393708,-0.006532,0.396597,...,-0.476816,-0.375111,-0.050547,-0.150825,0.003247,-0.004115,0.010272,0.006140,0,1.986376e-07
123621,123621,0.709646,-0.899855,-0.866564,-0.929319,-0.708629,0.548784,-0.350078,-0.008730,-0.208550,...,-0.209555,0.716978,0.866254,-0.565128,0.271600,-0.007365,0.013689,0.005739,0,3.014050e-04


In [20]:
test_flpr_just = test_flpr[['outlier_flag', 'outlier_probability']]
test_flpr_just.columns = ['outlier_flag_ver_5', 'outlier_probability_ver_5']
ketsu_test = pd.concat([test, test_flpr_just], axis = 1).drop(test.columns[0],axis=1)

In [21]:
ketsu_test

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_purchase_date_x,hist_latest_amount,hist_purchase_date_y,...,rate_new/hist_purchase_amount_max,rate_new/hist_purchase_amount_mean,outlier_flag_ver_1,outlier_flag_ver_2,outlier_flag_ver_3,outlier_probability_ver_3,outlier_flag_ver_4,outlier_probability_ver_4,outlier_flag_ver_5,outlier_probability_ver_5
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,306.0,24.0,1.513186e+09,-0.476731,1.503441e+09,...,-6.388255,1.120009,0,0,0,2.074851e-03,0,5.672927e-06,0,1.534506e-04
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,396.0,1.0,1.494079e+09,-0.731881,1.494079e+09,...,0.691844,0.902444,0,0,0,9.908335e-06,0,2.005090e-17,0,3.860746e-05
2,2017-08-01,C_ID_b709037bc5,5,1,1,184.0,4.0,1.517593e+09,0.214624,1.515681e+09,...,0.358015,0.066264,0,0,0,1.796332e-06,0,4.571899e-10,0,2.737026e-04
3,2017-12-01,C_ID_d27d835a9f,2,1,0,62.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,3.490204e-11,0,1.179702e-35,0,2.927275e-07
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,793.0,23.0,1.514992e+09,-0.596643,1.491330e+09,...,39.511246,-2.972271,0,0,0,3.157762e-16,0,0.000000e+00,0,1.227157e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,2,2,0,123.0,3.0,1.518957e+09,-0.705870,1.513517e+09,...,NaN,NaN,0,0,0,1.023316e-07,0,3.720316e-11,0,1.652304e-07
123619,2017-09-01,C_ID_75ace375ae,3,1,1,153.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0,0,0,2.853621e-06,0,8.862523e-11,0,4.465608e-07
123620,2016-09-01,C_ID_21d56d950c,5,1,1,518.0,1.0,1.506011e+09,0.369802,1.506011e+09,...,NaN,NaN,0,0,0,6.233753e-08,0,2.598190e-15,0,1.986376e-07
123621,2017-06-01,C_ID_6c46fc5a9d,2,1,0,245.0,19.0,1.506362e+09,-0.737892,1.500295e+09,...,0.999174,0.969611,0,0,0,5.024630e-03,0,2.783062e-12,0,3.014050e-04


In [22]:
ketsu.to_csv('../data/processed/processed20240626_train.csv',index=None)
ketsu_test.to_csv('../data/processed/processed20240626_test.csv',index=None)